# Persistent Cache

In [ ]:
import os.path, struct, tqdm
css = ""
if os.path.isfile("style.html"):
    from IPython.core.display import HTML
    with open("style.html", "r") as file:
        css = file.read()
HTML(css)

## Cache
Die Klasse `Cache` implementiert eine Transpositionstabelle die persistiert werden kann.

In [ ]:
class Cache:
    
    def __init__(self, min_limit = 0):
        self.min_limit = min_limit
        self.max_size = 100_000
        self.cache = {}

Die Methode `convert_state_to_bytes(state)` konvertiert einen Zustand in ein Byte-Array der Länge 7.

In [ ]:
def convert_state_to_bytes(state):
    byte_data = ((state[0][0] << 4) | state[0][1]).to_bytes(1,'big')
    for player in ['w','b']:
        for ring in range(2,-1,-1):
            ring_byte = 0
            for cell in range(7,-1,-1):
                ring_byte <<= 1
                if state[1][ring][cell] is player:
                    ring_byte |= 1
            byte_data += ring_byte.to_bytes(1,'big')
    return byte_data

Die Methode `write(...)` schreibt einen Zustand in den Cache. Dabei werden folgende Argumente erwartet:
- `state` $\in States$;
- `player` $\in Player$.
- `limit` $\in \mathbb{N}_0$;
- `value` $\in \mathopen[-1.0,1.0\mathclose]$;
- `alpha` $\in \mathopen[-1.0,1.0\mathclose]$;
- `beta` $\in \mathopen[-1.0,1.0\mathclose]$;

In [ ]:
def write(self, state, player: str, limit: int, value: float, alpha: float, beta: float) -> None:
    if limit < self.min_limit:
        return
    state = convert_state_to_bytes(state)
    key = state + (player == 'w').to_bytes(1,'big') + limit.to_bytes(1,'big')
    value = struct.pack("d", value) + struct.pack("d",alpha) + struct.pack("d",beta)
    self.cache[key] = value

Cache.write = write
del write

Die Methode `read(...)` liest einen vorher gespeicherten Zustand aus dem Cache aus. Falls der Zustand nicht vorhanden ist wird `None` zurück gegeben. Folgende Argumente werden erwartet:
- `state` $\in States$;
- `player` $\in Player$.
- `limit` $\in \mathbb{N}_0$;

Zurückgegeben wird ein Tripel bestehend aus:
1. `value` $\in \mathopen[-1.0,1.0\mathclose]$;
2. `alpha` $\in \mathopen[-1.0,1.0\mathclose]$;
3. `beta` $\in \mathopen[-1.0,1.0\mathclose]$;

In [ ]:
def read(self, state, player: str, limit: int) -> (float,float,float):
    state = convert_state_to_bytes(state)
    key = state + (player == 'w').to_bytes(1,'big') + limit.to_bytes(1,'big')
    value = self.cache.get(key)
    return (
        struct.unpack("d", value[:8])[0],
        struct.unpack("d", value[8:16])[0],
        struct.unpack("d", value[16:24])[0],
    ) if value else None

Cache.read = read
del read

Die Methode `clean()` prüft ob der Cache seine maximale Größe überschritten hat. Ist dies der Fall wird das minimale Limit um eins erhöht und alle Einträge, deren Limit geringer ist, werden aus dem Cache entfernt.

In [ ]:
def clean(self):
    while len(self.cache) > self.max_size:
        self.min_limit += 1
        pre_len = len(self.cache)
        to_delete = []
        for key,value in self.cache.items():
            limit = int(str(key[8]),16)
            if limit < self.min_limit:
                to_delete.append(key)
        for key in to_delete:
            self.cache.pop(key)
        print(f"Increased min_limit to {self.min_limit} and deleted {len(to_delete)} entries. Cache is now {len(self.cache)} entries big.")

Cache.clean = clean
del clean

Die Methode `save(...)` persistiert den Cache auf dem Dateisystem des Computers. Dafür wird folgender Paramter erwartet:
- `path` beschreibt den Pfad zur Cache-Datei im Dateisystem;

In [ ]:
def save(self, path: str):
    with open(path, "wb") as file:
        for key,value in tqdm.tqdm(self.cache.items()):
            file.write(key)
            file.write(value)
Cache.save = save
del save

Die Methode `load(...)` lädt einen vorhandenen Cache basierend auf einer Cache-Datei, die sich auf dem Dateisystem des Computers befindet. Dafür wird folgender Paramter erwartet:
- `path` beschreibt den Pfad zur Cache-Datei im Dateisystem;

In [ ]:
def load(self, path: str):
    if not os.path.isfile(path):
        return
    with open(path, "rb") as file:
        while True:
            key = file.read(9)
            value = file.read(24)
            if not key or not value:
                break
            self.cache[key] = value

Cache.load = load
del load